In [1]:
import json
import pandas as pd
import numpy as np
import os
import ember
from tqdm import tqdm

# Preprocessing

In [2]:
ember_folder = "../../../ember_dataset/"
train_ls = []
test_ls = []
#  Read EMBER dataset
extractor = ember.PEFeatureExtractor(2)
for root, dir, files in os.walk(ember_folder):
    for d in dir:
        if d == "ember":
            continue
        dir_path = os.path.join(root, d)
        for file in tqdm(os.listdir(dir_path)):
            if "train" in file:
                output = train_ls
            else:
                output = test_ls
            file_path = os.path.join(dir_path, file)
            print(file_path)
            data = pd.read_json(file_path, lines = True)

            filter = data[data["label"] != -1]
            # # X = np.array([])
            X = filter.apply(lambda x: extractor.process_raw_features(x), axis = 1)
            X_ = np.stack(X)
            Y = np.array(filter["label"])
            output.append(X_)
            output.append(Y)
            # X_total = np.concatenate(X_train_ls, axis=0)
            # Y_total = np.concatenate(Y_train_ls, axis=0)
            # output.append(data)
            # break
        break

  0%|          | 0/7 [00:00<?, ?it/s]

../../../ember_dataset/ember2017\train_features_1.jsonl


 14%|█▍        | 1/7 [01:49<10:58, 109.75s/it]

../../../ember_dataset/ember2017\train_features_0.jsonl


 14%|█▍        | 1/7 [02:15<13:31, 135.28s/it]


KeyboardInterrupt: 

In [3]:
len(train_ls[0::2])

1

In [4]:
for i in range(len(train_ls)):
    print(train_ls[i].shape)

(110000, 2381)
(110000,)


In [5]:
print(pd.__version__)

1.1.5


In [6]:
# X_train_ls = []
# Y_train_ls = []
# for train in train_ls:
#     filter = filter[filter["label"] != -1]
#     # X = np.array([])
#     extractor = ember.PEFeatureExtractor(2)
#     X = filter.apply(lambda x: extractor.process_raw_features(x), axis = 1)
#     X_ = np.stack(X)
#     Y = np.array(filter["label"])
#     X_train_ls.append(X_)
#     Y_train_ls.append(Y)
X_total = np.concatenate(train_ls[0::2], axis=0)
Y_total = np.concatenate(train_ls[1::2], axis=0)

In [7]:
np.savez("ember_subset_vectos.npz", X_total, Y_total)

In [8]:
# X_total = np.concatenate(ls, axis = 0)
X_total.shape

(110000, 2381)

In [9]:
X_.shape, Y.shape

((110000, 2381), (110000,))

# Training

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib
import pickle

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_, Y, test_size=0.2, random_state=42, stratify=Y)

In [12]:
print("Number of used features:", X_train.shape[1])

Number of used features: 2381


In [13]:
clf = RandomForestClassifier(
    # Set the number of trees to 100
    n_estimators=100,
    # Set the random state to 0 to ensure reproducibility
    random_state=0,
    # Enable the out-of-bag (OOB) score
    oob_score = True,
    # Set the maximum depth of the trees to 16
    max_depth = 16)

# Fit the classifier to the training data
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=16, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=True, random_state=0, verbose=0,
                       warm_start=False)

In [14]:
y_pred = clf.predict(X_test)

In [15]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(report)

Accuracy: 0.9765454545454545
              precision    recall  f1-score   support

           0       0.98      0.97      0.97     10000
           1       0.98      0.98      0.98     12000

    accuracy                           0.98     22000
   macro avg       0.98      0.98      0.98     22000
weighted avg       0.98      0.98      0.98     22000



In [16]:
joblib.dump(clf, 'rf_ember_subset.joblib')
# sel_features = X_train.columns
# open('../models/features.pkl', 'wb').write(pickle.dumps(sel_features)) 

['rf_ember_subset.joblib']